In [ ]:
import re
import emoji
import contractions
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from autocorrect import Speller
import numpy as np

In [ ]:
class LexicalEmbeddingPipeline:
    def __init__(self):
        """
        Pipeline para gerar representações léxicas para textos, baseadas em TF-IDF, contagens e outras métricas.
        """
        self.spell = Speller(lang='en')
        self.slang_map = {
            "asap": "as soon as possible",
            "idk": "i do not know",
            "lol": "laughing out loud"
        }
        self.stemmer = PorterStemmer()
        self.stop_words = set(stopwords.words('english'))

        # Vetorizadores
        self.tfidf_vectorizer = None
        self.count_vectorizer = None

    def preprocess_text(self, text):
        """Pré-processa o texto para limpeza e normalização."""
        # Correção ortográfica
        text = self.spell(text)

        # Substituir emojis por descrições (demojify)
        text = emoji.replace_emoji(text, replace=lambda e: f" {emoji.demojize(e)} ")

        # Corrigir quebras de linha e remover pontuação
        text = re.sub(r'[\n\t]', ' ', text)
        text = re.sub(r'[^\w\s]', '', text)

        # Remover acentos
        text = re.sub(r'[\u0300-\u036f]', '', text)

        # Expandir contrações
        text = contractions.fix(text)

        # # Expandir gírias
        # text = ' '.join([self.slang_map.get(word.lower(), word) for word in text.split()])

        # Remover links, hashtags, XMLs, etc.
        text = re.sub(r'http\S+|www\S+|\w+\.\w{2,3}|#[\w]+|<[^>]+>', '', text)

        # Remover stop words e aplicar stemming -> ou lemmas ? 
        tokens = [self.stemmer.stem(word) for word in text.split() if word.lower() not in self.stop_words]

        return tokens

    def fit_vectorizers(self, corpus):
        """Treina os vetorizadores TF-IDF e Count Vectorizer no corpus."""
        self.tfidf_vectorizer = TfidfVectorizer(max_features=5000)
        self.count_vectorizer = CountVectorizer(max_features=5000)

        # Ajustar vetorizadores no corpus
        self.tfidf_vectorizer.fit(corpus)
        self.count_vectorizer.fit(corpus)

    def generate_word_embeddings(self, text):
        """Gera embeddings para cada palavra no texto pré-processado."""
        # Pré-processar o texto -> já foi feito antes, mas tem q lembrar q isso vai ser usado tbm depois pra teste !!!
        tokens = self.preprocess_text(text)

        # Obter valores TF-IDF e Count para cada token
        tfidf_values = self.tfidf_vectorizer.transform([" ".join(tokens)]).toarray()[0]
        count_values = self.count_vectorizer.transform([" ".join(tokens)]).toarray()[0]

        token_embeddings = []
        for token in tokens:
            tfidf_value = tfidf_values[self.tfidf_vectorizer.vocabulary_.get(token, 0)]
            count_value = count_values[self.count_vectorizer.vocabulary_.get(token, 0)]

            # Concatenação de valores (sugestões adicionais abaixo)
            embedding = np.array([tfidf_value, count_value])
            token_embeddings.append(embedding)

        return tokens, np.array(token_embeddings)

    def process_corpus(self, corpus):
        """Processa o corpus completo, aplicando limpeza e gerando embeddings para cada palavra."""
        processed_corpus = [" ".join(self.preprocess_text(text)) for text in corpus]

        # Treinar vetorizadores no corpus completo
        self.fit_vectorizers(processed_corpus)

        # Gerar embeddings por texto
        results = []
        for text in corpus:
            tokens, embeddings = self.generate_word_embeddings(text)
            results.append({"tokens": tokens, "embeddings": embeddings})

        return results

In [ ]:
# Exemplo de corpus
corpus = [
        "I love natural language processing! 😊", 
        "ASAP, I will learn machine learning techniques. #AI",
        "TF-IDF and Bag-of-Words are great for lexical analysis."
]

# Inicializar pipeline
pipeline = LexicalEmbeddingPipeline()

# Processar corpus
results = pipeline.process_corpus(corpus)

# Exibir resultados
for i, result in enumerate(results):
    print(f"Texto {i+1} Tokens: {result['tokens']}")
    print(f"Texto {i+1} Embeddings:\n{result['embeddings']}\n")
